In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns    
%matplotlib inline

In [3]:
data_train = pd.DataFrame(pd.read_csv("./feature_3.csv",header = 0))

In [5]:
import xgboost as xgb

#去除不要的行

drop_cols = ['User_id','Merchant_id','Coupon_id','Discount_rate','Date_received','Date','coupon_not_use','not_buy'
            ,'count_2', 'count_3', 'count_4']
data_train_r = data_train.drop(columns = drop_cols)
X_train = data_train_r.drop(columns = ['coupon_use'])
Y_train = data_train_r.loc[:,'coupon_use'].values

In [6]:
from xgboost import XGBClassifier
cls_model = XGBClassifier(max_depth=8, # 构建树的深度，越大越容易过拟合
                      n_estimators=5000, # 最佳迭代次数
                      learning_rate=0.1, # 学习率
                      scale_pos_weight=1,  # 解决样本个数不平衡的问题
                      n_jobs=-1)
cls_model.fit(X_train, # 训练集
              Y_train, # 训练集标签
              eval_set=[(X_train,Y_train)], # 验证集
              verbose = 10, # 训练多少轮打印一次结果
              early_stopping_rounds=10 # 每经过10轮rmse必须下降，否则停止训练
             )

[0]	validation_0-error:0.021189
Will train until validation_0-error hasn't improved in 10 rounds.
[10]	validation_0-error:0.019363
[20]	validation_0-error:0.018808
[30]	validation_0-error:0.018503
[40]	validation_0-error:0.018309
[50]	validation_0-error:0.018101
[60]	validation_0-error:0.017933
[70]	validation_0-error:0.017816
[80]	validation_0-error:0.01767
[90]	validation_0-error:0.017589
[100]	validation_0-error:0.017456
[110]	validation_0-error:0.017373
[120]	validation_0-error:0.017234
[130]	validation_0-error:0.017142
[140]	validation_0-error:0.017028
[150]	validation_0-error:0.016891
[160]	validation_0-error:0.016748
[170]	validation_0-error:0.016635
[180]	validation_0-error:0.016541
[190]	validation_0-error:0.016443
[200]	validation_0-error:0.016341
[210]	validation_0-error:0.016258
[220]	validation_0-error:0.016189
[230]	validation_0-error:0.016098
[240]	validation_0-error:0.016004
[250]	validation_0-error:0.015955
[260]	validation_0-error:0.015871
[270]	validation_0-error:0.0

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=8, min_child_weight=1, missing=None, n_estimators=5000,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [7]:
#读取测试文件
data_test = pd.DataFrame(pd.read_csv("./ccf_offline_stage1_test_revised.csv",header = 0))
#data_test.duplicated().value_counts()
#得到券的折扣率，无券为0
data_test['Discount_rate_num'] =data_test['Discount_rate'].isnull().apply(lambda x: 0 if x else 1)
x = []
for i,j in zip(data_test['Discount_rate_num'], data_test['Discount_rate']):
    #print(i,j)
    if i == 0:
        x.append(0)
    else:
        a = j.split(':')
        a = list(map(float, a))
        if len(a) < 2:
            x.append(a[0])
        else:
            x.append(a[1]/a[0])
    
    
data_test['Discount_rate_num'] = x
#data_test
#print(data_test['Discount_rate_num'])

#得到用户的券的满减区间 0-50 为 ‘2’ 50-200 为‘3’ 200-500 为‘4’ 直接折扣为 ‘1’

data_test['Discount_range'] =data_test['Discount_rate'].isnull().apply(lambda x: 0 if x else 1)
x = []
for i,j in zip(data_test['Discount_range'], data_test['Discount_rate']):
    #print(i,j)
    if i == 0:
        x.append('0')
    else:
        a = j.split(':')
        a = list(map(float, a))
        if len(a) < 2:
            x.append('1')
        else:
            if a[0] <= 50:
                x.append('2')
            elif a[0] <= 200:
                x.append('3')
            else:
                x.append('4')
    
    
data_test['Discount_range'] = x
#data_test

#用户信息合并列
cols = ['User_id','total_count','coupon_count','buy_count','use_count','coupon_not_use_count','not_buy_count']
user_train_merge = data_train[cols]
user_train_merge = user_train_merge.drop_duplicates()

#合并
data_test_merge = pd.merge(data_test,user_train_merge,on='User_id',how='left')

#商家信息合并列
cols = ['Merchant_id','M_total_count','M_coupon_count','M_buy_count','M_use_count','M_not_use_count','M_not_buy_count','M_use_rate']
m_train_merge = data_train[cols]
m_train_merge = m_train_merge.drop_duplicates()

#合并
data_test_merge1 = pd.merge(data_test_merge,m_train_merge,on='Merchant_id',how='left')

#商家信息合并列
cols = ['Coupon_id','C_total_count','C_use_count','C_not_use_count','C_use_rate']
#data_train['Coupon_id'] = 
data_train['Coupon_id'] = data_train['Coupon_id'].replace(np.nan, 0)
data_train['Coupon_id'] = data_train[['Coupon_id']].astype(int)
data_train['Coupon_id'] = data_train['Coupon_id'].replace(0, np.nan)
c_train_merge = data_train[cols]
c_train_merge = c_train_merge.drop_duplicates()
#合并
data_test_merge2 = pd.merge(data_test_merge1,c_train_merge,on='Coupon_id',how='left')

cols = ['User_id', '2_use_count', '3_use_count','4_use_count','2_use','3_use','4_use','2_use_rate','3_use_rate','4_use_rate']
user_train_merge = data_train[cols]
user_train_merge = user_train_merge.drop_duplicates()

#合并
data_test_merge3 = pd.merge(data_test_merge2,user_train_merge,on='User_id',how='left')

cols = ['User_id', '2_count' ,'use_rate', '4_count', '3_count']
user_train_merge = data_train[cols]
user_train_merge = user_train_merge.drop_duplicates()

#合并
data_test_merge4 = pd.merge(data_test_merge3,user_train_merge,on='User_id',how='left')

data_test_merge4.duplicated().value_counts()
data_test_merge4.columns

cols = X_train.columns.tolist()
data_test_merge4['Discount_range'] = data_test_merge4['Discount_range'].astype(int)



NameError: name 'reg_model' is not defined

In [8]:
y_pred=cls_model.predict(data_test_merge4[cols])

data_test['Probability'] = y_pred
data_test['Probability'] = data_test['Probability'].apply(lambda x: x if x >= 0.0 else 0.0 )
data_test['Probability'] = data_test['Probability'].apply(lambda x: x if x <= 1.0 else 1.0 )
data_test['Probability'].max()

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


1

In [9]:
res_cols = ['User_id', 'Coupon_id', 'Date_received', 'Probability']
data_test[res_cols].to_csv('./res_5.csv',index=False,header = 0)

In [10]:
data_test_merge4.to_csv('./test_feature_5.csv', index = False)